In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import nltk
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import string
import re
from sklearn.svm import SVC
import wordcloud
import string
string.punctuation
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from sklearn.metrics import classification_report, confusion_matrix

# Overview

Spam messages are unimportant messages like advertisements for goods and services or business opportunities delivered to a big group of people without their permission. The project aims at classifying these messages as ham or spam for optimizing user effort. 

# Data Loading

In [ ]:
sms=pd.read_csv('/kaggle/input/sms-spam-collection-dataset/spam.csv')
sms.head()

In [ ]:
sns.countplot(sms['v1'])

The dataset contains 13.4% spam and 86.6% ham.

In [ ]:
# Percentage of data points with ham label 
print(sms[sms['v1']=='ham'].shape[0]*100/sms.shape[0])

# EDA

In [ ]:
sms.info()

In [ ]:
sms.describe()

In [ ]:
sms.isnull().sum()

In [ ]:
sms=sms.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1)
sms.isnull().sum()

In [ ]:
sms=sms.rename({'v1':'label','v2':'text'},axis=1)
sms.head()

Distribution based on length of words

In [ ]:
sms['length']=sms['text'].apply(lambda x: len(x))
sns.distplot(sms['length'], kde=True)

In [ ]:
sns.boxplot(y='length', x='label', data=sms)

In [ ]:
dt=sms.groupby('label').mean()
#sns.barplot(y='length', x='label', data=dt)
dt

In [ ]:
sms.groupby('label').describe()

In [ ]:
sms.hist(column='length', by='label', bins=50,figsize=(20,8))

In [ ]:
ham=sms[sms['label']=='ham'].copy()
spam=sms[sms['label']=='spam'].copy()

In [ ]:
def show_wordcloud(df):
    text = ' '.join(df['text'].astype(str).tolist())
    stopwords = set(wordcloud.STOPWORDS)
    diag = wordcloud.WordCloud(stopwords=stopwords,background_color='lightblue',
                    colormap='coolwarm', width=800, height=600).generate(text)
    
    
    plt.figure(figsize=(10,7), frameon=True)
    plt.imshow(diag)  
    plt.show()

show_wordcloud(ham)

In [ ]:
show_wordcloud(spam)

In [ ]:
ham.head()

In [ ]:
def word_count_plot(x):
    a = [i for i in x if i not in string.punctuation]
    a = "".join(a).split()
    b = [word.lower() for word in a if word.lower() not in stopwords.words("english")]  
    return b

In [ ]:
ham['text'] = ham['text'].apply(word_count_plot)
words_ham = ham['text'].tolist()
spam['text'] = spam['text'].apply(word_count_plot)
words_spam = spam['text'].tolist()

In [ ]:
ham.head()

In [ ]:
ham_words = []
for i in words_ham:
    for item in i:
        ham_words.append(item)
spam_words = []
for i in words_spam:
    for item in i:
        spam_words.append(item)

In [ ]:
from collections import Counter
ham_count  = Counter(ham_words)
df_ham  = pd.DataFrame(ham_count.most_common(10),  columns=['word', 'count'])
spam_count = Counter(spam_words)
df_spam = pd.DataFrame(spam_count.most_common(10), columns=['word', 'count'])

Top 10 ham words

In [ ]:
plt.figure(figsize=(10,7))
sns.barplot(x='word', y='count', data=df_ham)

Top 10 spam words

In [ ]:
plt.figure(figsize=(10,7))
sns.barplot(x='word', y='count', data=df_spam)

# Data Preprocessing

Removing stopwords like 'where', 'why', 'how', 'all', 'any', 'both', 'each' and punctuation symbols is essential before text based predictions. These symbols have an impact on the prediction accuracy if not removed.

Stemming is the process of producing morphological variants of a root/base word. A stemming algorithm reduces the words “chocolates”, “chocolatey”, “choco” to the root word, “chocolate” and “retrieval”, “retrieved”, “retrieves” reduce to the stem “retrieve”. Porter stemmer helps us stem text data used.

In [ ]:
ps=PorterStemmer()
def preprocess(x):
    x = x.lower()
    x = re.sub(r'[^0-9a-zA-Z]', ' ', x)
    x = re.sub(r'\s+', ' ', x)
    l = [item for item in x if item not in string.punctuation]
    processed=[ps.stem(i) for i in l if i not in stopwords.words('english')]
    return processed


In [ ]:
X=sms['text']
y=sms['label']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

# Pipeline 

Bag of words (BOW) / Bag of n-grams 

By using the bag-of-words technique, we convert a text into its equivalent vector of numbers. It involves 3 sub processes:
1. Tokenization - tokenizing strings and giving an integer id for each possible token by using white-spaces and punctuation as token separators.
2. Vectorization - counting the occurrences of tokens in each document.
3. TF-IDF - normalizing and weighting with diminishing importance tokens that occur in the majority of documents.

In [ ]:
def model(model_name,X_train,y_train,X_test,y_test):
    pipeline=Pipeline([
    ('bow', CountVectorizer(analyzer=preprocess)),
    ('tfidf', TfidfTransformer()),
    ('model', model_name),
    ])
    pipeline.fit(X_train,y_train)

    preds=pipeline.predict(X_test)

    print (classification_report(y_test,preds))
    print (confusion_matrix(y_test,preds))
    print('Accuracy:', pipeline.score(X_test, y_test)*100)
    print("Training Score:",pipeline.score(X_train,y_train)*100)
    from sklearn.metrics import accuracy_score
    score = accuracy_score(y_test,preds)
    return score
    


# MultinomialNB 

In [ ]:
mnb=model(MultinomialNB(),X_train,y_train,X_test,y_test)

# RF

In [ ]:
rf=model(RandomForestClassifier(),X_train,y_train,X_test,y_test)

# Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb=model(GradientBoostingClassifier(),X_train,y_train,X_test,y_test)

# XGBoost

In [ ]:
xgb=model(XGBClassifier(),X_train,y_train,X_test,y_test)

# SVC

In [ ]:
svc=model(SVC(),X_train,y_train,X_test,y_test)

# Model Comparison 

In [ ]:
models = pd.DataFrame({
    'Model':['MNB','RF', 'GBoost', 'XGBoost', 'SVC'],
    'Accuracy_score' :[mnb ,rf, gb, xgb, svc]
})
sns.barplot(x='Accuracy_score', y='Model', data=models)
models.sort_values(by='Accuracy_score', ascending=False)